## Date Handling of Input Files

> The data contains periods in CET/CEST format.
	

* we need to have start and end date in one consitent time zone
* we need to split the period into start and end date of the period 

"01/01/2015 00:00:00 - 01/01/2015 00:15:00"


In [28]:
# "31/03/2024 01:15:00 - 31/03/2024 01:30:00"			
# "31/03/2024 01:30:00 - 31/03/2024 01:45:00 (CET)"		
# "31/03/2024 01:45:00 (CET) - 31/03/2024 03:00:00 (CEST)"	
# "31/03/2024 03:00:00 (CEST) - 31/03/2024 03:15:00"		
# "31/03/2024 03:15:00 - 31/03/2024 03:30:00"			

# "27/10/2024 01:45:00 - 27/10/2024 02:00:00 (CEST)"		
# "27/10/2024 02:00:00 (CEST) - 27/10/2024 02:15:00 (CEST)"	
# "27/10/2024 02:15:00 (CEST) - 27/10/2024 02:30:00 (CEST)"	
# "27/10/2024 02:30:00 (CEST) - 27/10/2024 02:45:00 (CEST)"	
# "27/10/2024 02:45:00 (CEST) - 27/10/2024 02:00:00 (CET)"	
# "27/10/2024 02:00:00 (CET) - 27/10/2024 02:15:00 (CET)"		
# "27/10/2024 02:15:00 (CET) - 27/10/2024 02:30:00 (CET)"		
# "27/10/2024 02:30:00 (CET) - 27/10/2024 02:45:00 (CET)"		
# "27/10/2024 02:45:00 (CET) - 27/10/2024 03:00:00"		
# "27/10/2024 03:00:00 - 27/10/2024 03:15:00"			


# "26/03/2016 23:00:00 - 27/03/2016 00:00:00"			
# "27/03/2016 00:00:00 - 27/03/2016 01:00:00 (CET)"		
# "27/03/2016 01:00:00 (CET) - 27/03/2016 03:00:00 (CEST)"	
# "27/03/2016 03:00:00 (CEST) - 27/03/2016 04:00:00"		
# "27/03/2016 04:00:00 - 27/03/2016 05:00:00"			

# "30/10/2016 00:00:00 - 30/10/2016 01:00:00"			
# "30/10/2016 01:00:00 - 30/10/2016 02:00:00 (CEST)"		
# "30/10/2016 02:00:00 (CEST) - 30/10/2016 02:00:00 (CET)"	
# "30/10/2016 02:00:00 (CET) - 30/10/2016 03:00:00"		
# "30/10/2016 03:00:00 - 30/10/2016 04:00:00"		

## Setup

All the manipulations and plots in this notebook can be created with standard libraries such as matplotlib, statsmodels etc. 

In [29]:
# Main data packages. 
import numpy as np
import pandas as pd

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': 'white', 'grid.color': '.8'}
)
NF_ORANGE = '#ff5a36'
NF_BLUE = '#163251'
cmaps_hex = ['#193251','#FF5A36','#696969', '#7589A2','#FF5A36', '#DB6668']
sns.set_palette(palette=cmaps_hex)
sns_c = sns.color_palette(palette=cmaps_hex)
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [30]:
sns_c

[(0.09803921568627451, 0.19607843137254902, 0.3176470588235294),
 (1.0, 0.35294117647058826, 0.21176470588235294),
 (0.4117647058823529, 0.4117647058823529, 0.4117647058823529),
 (0.4588235294117647, 0.5372549019607843, 0.6352941176470588),
 (1.0, 0.35294117647058826, 0.21176470588235294),
 (0.8588235294117647, 0.4, 0.40784313725490196)]

## Import Data 

The data for this notebook was downloaded from the [meteoblue website](https://www.meteoblue.com/en/weather/archive/export/basel_switzerland_2661604) and consits of weather data for the city of Basel from 2008 till 2020. 

In [31]:
files = [
    "../../data/da_prices/original/GUI_ENERGY_PRICES_201412312300-201512312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_201512312300-201612312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_201612312300-201712312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_201712312300-201812312300 - 1.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_201712312300-201812312300 - 2.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_201812312300-201912312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_201912312300-202012312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_202012312300-202112312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_202112312300-202212312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_202212312300-202312312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_202312312300-202412312300.csv",
    "../../data/da_prices/original/GUI_ENERGY_PRICES_202412312300-202512312300.csv",
]
raw_df = pd.concat((pd.read_csv(f, delimiter=",") for f in files), ignore_index=True)


In [32]:
raw_df.shape  #(287067, 8)

(541275, 6)

In [33]:
raw_df.columns

Index(['MTU (CET/CEST)', 'Area', 'Sequence', 'Day-ahead Price (EUR/MWh)',
       'Intraday Period (CET/CEST)', 'Intraday Price (EUR/MWh)'],
      dtype='object')

'MTU (CET/CEST) End UTC': 'timestamp_UTC',
'price' #'da_price_eur_mwh'

In [34]:
# Create working copy of dataframe
df_utc_op = raw_df.copy()
# select only those values
df_utc_op = df_utc_op[df_utc_op["Sequence"] != "Sequence Sequence 2"]
 # drop columns
df_utc_op.drop(columns=["Area","Sequence","Intraday Period (CET/CEST)","Intraday Price (EUR/MWh)"], inplace=True)
# Rename columns in a more pythonic way
df_utc_op = df_utc_op.rename(columns={
    'MTU (CET/CEST)': 'period',  
    'Day-ahead Price (EUR/MWh)' : 'price' # adjust   #'da_price_eur_mwh'
    }
)

df_utc_op.head(4)


,period,price
0,05/01/2015 00:00:00 - 05/01/2015 01:00:00,22.34
1,05/01/2015 01:00:00 - 05/01/2015 02:00:00,17.93
2,05/01/2015 02:00:00 - 05/01/2015 03:00:00,15.17
3,05/01/2015 03:00:00 - 05/01/2015 04:00:00,16.38


In [35]:
df_utc_op.shape

(287067, 2)

In [36]:
# Split "period" into start/end strings
df_utc_op[["period_start", "period_end"]] = df_utc_op["period"].str.split(" - ", n=1, expand=True)
df_utc_op.head()

,period,price,period_start,period_end
0,05/01/2015 00:00:00 - 05/01/2015 01:00:00,22.34,05/01/2015 00:00:00,05/01/2015 01:00:00
1,05/01/2015 01:00:00 - 05/01/2015 02:00:00,17.93,05/01/2015 01:00:00,05/01/2015 02:00:00
2,05/01/2015 02:00:00 - 05/01/2015 03:00:00,15.17,05/01/2015 02:00:00,05/01/2015 03:00:00
3,05/01/2015 03:00:00 - 05/01/2015 04:00:00,16.38,05/01/2015 03:00:00,05/01/2015 04:00:00
4,05/01/2015 04:00:00 - 05/01/2015 05:00:00,17.38,05/01/2015 04:00:00,05/01/2015 05:00:00


In [37]:

def add_timezone_and_utc(df, col): #="period_start"):
    # State: start in CET
    tz_state = "CET"
    tz_list = []

    # Walk rows in order and flip state if original string contains CET/CEST
    for val in df[col].astype(str):
        if "CEST" in val:
            tz_state = "CEST"
        elif "CET" in val:
            tz_state = "CET"
        tz_list.append(tz_state)

    # Parse datetime from the column (remove any existing timezone text)
    base_dt = pd.to_datetime(
        df[col].astype(str).str.replace(r"\s*\(.*\)$", "", regex=True),
        dayfirst=True,
        errors="coerce"
    )

    # Add labeled version
    # df["period_start_labeled"] = (
    df[col + "_labeled"] = (
        base_dt.dt.strftime("%d/%m/%Y %H:%M:%S") + " (" + pd.Series(tz_list, index=df.index) + ")"
    )

    # Compute UTC by subtracting 1 or 2 hours depending on CET/CEST
    offset_hours = pd.Series(tz_list, index=df.index).map({"CET": 1, "CEST": 2})
    df[col + "_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")
    # df["tsp_start_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")

    return df


In [38]:
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_start")
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_end")

In [39]:
df_utc_op

,period,price,period_start,period_end,period_start_labeled,period_start_utc,period_end_labeled,period_end_utc
0,05/01/2015 00:00:00 - 05/01/2015 01:00:00,22.34,05/01/2015 00:00:00,05/01/2015 01:00:00,05/01/2015 00:00:00 (CET),2015-01-04 23:00:00,05/01/2015 01:00:00 (CET),2015-01-05 00:00:00
1,05/01/2015 01:00:00 - 05/01/2015 02:00:00,17.93,05/01/2015 01:00:00,05/01/2015 02:00:00,05/01/2015 01:00:00 (CET),2015-01-05 00:00:00,05/01/2015 02:00:00 (CET),2015-01-05 01:00:00
2,05/01/2015 02:00:00 - 05/01/2015 03:00:00,15.17,05/01/2015 02:00:00,05/01/2015 03:00:00,05/01/2015 02:00:00 (CET),2015-01-05 01:00:00,05/01/2015 03:00:00 (CET),2015-01-05 02:00:00
3,05/01/2015 03:00:00 - 05/01/2015 04:00:00,16.38,05/01/2015 03:00:00,05/01/2015 04:00:00,05/01/2015 03:00:00 (CET),2015-01-05 02:00:00,05/01/2015 04:00:00 (CET),2015-01-05 03:00:00
4,05/01/2015 04:00:00 - 05/01/2015 05:00:00,17.38,05/01/2015 04:00:00,05/01/2015 05:00:00,05/01/2015 04:00:00 (CET),2015-01-05 03:00:00,05/01/2015 05:00:00 (CET),2015-01-05 04:00:00
...,...,...,...,...,...,...,...,...
541265,31/12/2025 22:45:00 - 31/12/2025 23:00:00,73.77,31/12/2025 22:45:00,31/12/2025 23:00:00,31/12/2025 22:45:00 (CET),2025-12-31 21:45:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00
541267,31/12/2025 23:00:00 - 31/12/2025 23:15:00,86.30,31/12/2025 23:00:00,31/12/2025 23:15:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00
541269,31/12/2025 23:15:00 - 31/12/2025 23:30:00,76.48,31/12/2025 23:15:00,31/12/2025 23:30:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00
541271,31/12/2025 23:30:00 - 31/12/2025 23:45:00,75.75,31/12/2025 23:30:00,31/12/2025 23:45:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00,31/12/2025 23:45:00 (CET),2025-12-31 22:45:00


In [40]:
df_utc_op.columns

Index(['period', 'price', 'period_start', 'period_end', 'period_start_labeled',
       'period_start_utc', 'period_end_labeled', 'period_end_utc'],
      dtype='object')

In [41]:
# Create working copy of dataframe
df_utc_op_temp = df_utc_op.copy()
# select only those values
# df_price = df_price[df_price["Sequence"] != "Sequence Sequence 2"]
 # drop columns
df_utc_op.drop(columns=['period_start', 'period_end', 'period_start_labeled', 'period_end_labeled'], inplace=True)

df_utc_op = df_utc_op.assign(
    date1=lambda x: x["period_start_utc"].dt.date,
    date=lambda x: pd.to_datetime(
        x["date1"],
        format="mixed",
        dayfirst=True,
        errors="coerce"
    ),
    year=lambda x: x["period_start_utc"].dt.year,
    month=lambda x: x["period_start_utc"].dt.month,
    #monthname=lambda x: x["period_start_utc"].dt.month_name(),
    day=lambda x: x["period_start_utc"].dt.day,
    dayofyear=lambda x: x["period_start_utc"].dt.dayofyear,
    hour=lambda x: x["period_start_utc"].dt.hour,
    #week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype(int),
    week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype("Int64"),
    dayofweek=lambda x: x["period_start_utc"].dt.dayofweek,   # 0=Mon ... 6=Sun
    #dayname=lambda x: x["period_start_utc"].dt.day_name(),   

)
df_utc_op.drop(columns=['date1'], inplace=True)

df_utc_op.head()

,period,price,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
0,05/01/2015 00:00:00 - 05/01/2015 01:00:00,22.34,2015-01-04 23:00:00,2015-01-05 00:00:00,2015-01-04,2015,1,4,4,23,1,6
1,05/01/2015 01:00:00 - 05/01/2015 02:00:00,17.93,2015-01-05 00:00:00,2015-01-05 01:00:00,2015-01-05,2015,1,5,5,0,2,0
2,05/01/2015 02:00:00 - 05/01/2015 03:00:00,15.17,2015-01-05 01:00:00,2015-01-05 02:00:00,2015-01-05,2015,1,5,5,1,2,0
3,05/01/2015 03:00:00 - 05/01/2015 04:00:00,16.38,2015-01-05 02:00:00,2015-01-05 03:00:00,2015-01-05,2015,1,5,5,2,2,0
4,05/01/2015 04:00:00 - 05/01/2015 05:00:00,17.38,2015-01-05 03:00:00,2015-01-05 04:00:00,2015-01-05,2015,1,5,5,3,2,0


In [42]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
Index: 287067 entries, 0 to 541273
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            287067 non-null  object        
 1   price             287067 non-null  float64       
 2   period_start_utc  287067 non-null  datetime64[ns]
 3   period_end_utc    287067 non-null  datetime64[ns]
 4   date              287067 non-null  datetime64[ns]
 5   year              287067 non-null  int32         
 6   month             287067 non-null  int32         
 7   day               287067 non-null  int32         
 8   dayofyear         287067 non-null  int32         
 9   hour              287067 non-null  int32         
 10  week              287067 non-null  Int64         
 11  dayofweek         287067 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), float64(1), int32(6), object(1)
memory usage: 22.2+ MB


In [52]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "hour", "week", "dayofweek"
]

df_utc_h = (
    df_utc_op
    .groupby(group_cols, as_index=False)
    .agg(
        price=("price", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_hour=("year", "size"),   # count rows per group
    )
)
df_utc_h.head()

,date,year,month,day,dayofyear,hour,week,dayofweek,price,period_start_utc,period_end_utc,c_by_hour
0,2015-01-04,2015,1,4,4,23,1,6,22.34,2015-01-04 23:00:00,2015-01-05 00:00:00,1
1,2015-01-05,2015,1,5,5,0,2,0,17.93,2015-01-05 00:00:00,2015-01-05 01:00:00,1
2,2015-01-05,2015,1,5,5,1,2,0,15.17,2015-01-05 01:00:00,2015-01-05 02:00:00,1
3,2015-01-05,2015,1,5,5,2,2,0,16.38,2015-01-05 02:00:00,2015-01-05 03:00:00,1
4,2015-01-05,2015,1,5,5,3,2,0,17.38,2015-01-05 03:00:00,2015-01-05 04:00:00,1


In [53]:
df_utc_h.tail()

,date,year,month,day,dayofyear,hour,week,dayofweek,price,period_start_utc,period_end_utc,c_by_hour
96331,2025-12-31,2025,12,31,365,18,1,2,95.9900,2025-12-31 18:00:00,2025-12-31 19:00:00,4
96332,2025-12-31,2025,12,31,365,19,1,2,86.8025,2025-12-31 19:00:00,2025-12-31 20:00:00,4
96333,2025-12-31,2025,12,31,365,20,1,2,79.7975,2025-12-31 20:00:00,2025-12-31 21:00:00,4
96334,2025-12-31,2025,12,31,365,21,1,2,81.3900,2025-12-31 21:00:00,2025-12-31 22:00:00,4
96335,2025-12-31,2025,12,31,365,22,1,2,78.5700,2025-12-31 22:00:00,2025-12-31 23:00:00,4


In [54]:
df_utc_h["c_by_hour"].value_counts()

c_by_hour
4    63577
1    32759
Name: count, dtype: int64

In [55]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "week", "dayofweek"
]

df_utc_d = (
    df_utc_h
    .groupby(group_cols, as_index=False)
    .agg(
        price=("price", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_day=("year", "size"),   # count rows per group
    )
)
df_utc_d.head()

,date,year,month,day,dayofyear,week,dayofweek,price,period_start_utc,period_end_utc,c_by_day
0,2015-01-04,2015,1,4,4,1,6,22.340000,2015-01-04 23:00:00,2015-01-05,1
1,2015-01-05,2015,1,5,5,2,0,36.387500,2015-01-05 00:00:00,2015-01-06,24
2,2015-01-06,2015,1,6,6,2,1,33.730833,2015-01-06 00:00:00,2015-01-07,24
3,2015-01-07,2015,1,7,7,2,2,38.795000,2015-01-07 00:00:00,2015-01-08,24
4,2015-01-08,2015,1,8,8,2,3,28.497083,2015-01-08 00:00:00,2015-01-09,24


In [56]:
df_utc_d.tail()

,date,year,month,day,dayofyear,week,dayofweek,price,period_start_utc,period_end_utc,c_by_day
4010,2025-12-27,2025,12,27,361,52,5,91.023333,2025-12-27,2025-12-28 00:00:00,24
4011,2025-12-28,2025,12,28,362,52,6,93.085417,2025-12-28,2025-12-29 00:00:00,24
4012,2025-12-29,2025,12,29,363,1,0,93.948021,2025-12-29,2025-12-30 00:00:00,24
4013,2025-12-30,2025,12,30,364,1,1,88.232292,2025-12-30,2025-12-31 00:00:00,24
4014,2025-12-31,2025,12,31,365,1,2,87.372609,2025-12-31,2025-12-31 23:00:00,23


In [57]:
df_utc_d["c_by_day"].value_counts()

c_by_day
24    4013
1        1
23       1
Name: count, dtype: int64

In [58]:
group_cols = [
    "year"
]

df_utc_y = (
    df_utc_d
    .groupby(group_cols, as_index=False)
    .agg(
        price=("price", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_year=("year", "size"),   # count rows per group
    )
)
df_utc_y.head(20)

,year,price,period_start_utc,period_end_utc,c_by_year
0,2015,32.051666,2015-01-04 23:00:00,2016-01-01 00:00:00,362
1,2016,29.136125,2016-01-01 00:00:00,2017-01-01 00:00:00,366
2,2017,34.414280,2017-01-01 00:00:00,2018-01-01 00:00:00,365
3,2018,44.699523,2018-01-01 00:00:00,2019-01-01 00:00:00,365
4,2019,37.876115,2019-01-01 00:00:00,2020-01-01 00:00:00,365
5,2020,30.659717,2020-01-01 00:00:00,2021-01-01 00:00:00,366
6,2021,97.394796,2021-01-01 00:00:00,2022-01-01 00:00:00,365
7,2022,236.643705,2022-01-01 00:00:00,2023-01-01 00:00:00,365
8,2023,95.684444,2023-01-01 00:00:00,2024-01-01 00:00:00,365
9,2024,78.978707,2024-01-01 00:00:00,2025-01-01 00:00:00,366


In [59]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["price"].isna()]

rows_with_missing_price

,period,price,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek


## saving our files by hour

In [60]:
df_utc_h.to_csv("../../data_cleaned/by_source/ENERGY_PRICES.csv", index=False)